# Hurricane Ike Maximum Water Levels
Compute the maximum water level during Hurricane Ike on a 9 million node triangular mesh storm surge model.  Plot the results using [HoloViz](https://holoviz.org/) TriMesh rendering with Datashader. 

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import fsspec
from dask.distributed import Client, progress, LocalCluster

### Start a dask cluster to crunch the data

In [ ]:
#cluster_type = 'local'
cluster_type = 'coiled'

In [ ]:
if cluster_type == 'coiled':
    import coiled
    cluster = coiled.Cluster(
        region="us-west-2",
        arm=True,
        worker_vm_types=["t4g.small"],  # cheap, small ARM instances, 2cpus, 2GB RAM
        worker_options={'nthreads':2},
        n_workers=30,
        wait_for_workers=True,
        compute_purchase_option="spot_with_fallback",
        software='esip-pangeo-arm',
        workspace='esip-lab',
        timeout=300   # leave cluster running for 5 min in case we want to use it again
    )

    client = cluster.get_client()

In [ ]:
if cluster_type=='local':
    cluster = LocalCluster()
    client = Client(cluster)

In [ ]:
if cluster_type=='gateway':
    from dask_gateway import Gateway
    gateway = Gateway()
    cluster = gateway.new_cluster()
    cluster.adapt(minimum=4, maximum=20);

In [ ]:
if cluster_type=='kubernetes':
    from dask_kubernetes import KubeCluster
    cluster = KubeCluster()
    cluster.adapt(minimum=4, maximum=20);

In [ ]:
client

### Read the data using the cloud-friendly zarr data format

In [ ]:
fs = fsspec.filesystem('s3', anon=True, endpoint_url='https://mghp.osn.xsede.org')

In [ ]:
ds = xr.open_dataset(fs.get_mapper('s3://rsignellbucket1/esip/adcirc/ike'), engine='zarr', chunks={'time':40})

In [ ]:
ds['zeta']

How many GB of sea surface height data do we have?

In [ ]:
ds['zeta'].nbytes/1.e9

Take the maximum over the time dimension.  This is the computationally intensive step.

In [ ]:
%%time
max_var = ds['zeta'].max(dim='time').compute()

### Visualize data on mesh using HoloViz.org tools

In [ ]:
import numpy as np
import datashader as dshade
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import hvplot.xarray
import holoviews.operation.datashader as dshade

dshade.datashade.precompute = True
hv.extension('bokeh')

In [ ]:
v = np.vstack((ds['x'], ds['y'], max_var)).T
verts = pd.DataFrame(v, columns=['x','y','vmax'])

In [ ]:
points = gv.operation.project_points(gv.Points(verts, vdims=['vmax']))

In [ ]:
tris = pd.DataFrame(ds['element'].values.astype('int')-1, columns=['v0','v1','v2'])

In [ ]:
tiles = gv.tile_sources.OSM

In [ ]:
value = 'max water level'
label = '{} (m)'.format(value)
trimesh = gv.TriMesh((tris, points), label=label)
mesh = dshade.rasterize(trimesh).opts(
              cmap='rainbow', colorbar=True, width=600, height=400)

In [ ]:
tiles * mesh

### Extract a time series at a specified lon, lat location

Because Xarray does not yet understand that `x` and `y` are coordinate variables on this triangular mesh, we create our own simple function to find the closest point. If we had a lot of these, we could use a more fancy tree algorithm.

In [ ]:
# find the indices of the points in (x,y) closest to the points in (xi,yi)
def nearxy(x,y,xi,yi):
    ind = np.ones(len(xi),dtype=int)
    for i in range(len(xi)):
        dist = np.sqrt((x-xi[i])**2+(y-yi[i])**2)
        ind[i] = dist.argmin()
    return ind

In [ ]:
#just offshore of Galveston
lat = 29.2329856
lon = -95.1535041

In [ ]:
ind = nearxy(ds['x'].values,ds['y'].values,[lon], [lat])

In [ ]:
ds['zeta'][:,ind].hvplot(x='time', grid=True)